In [641]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Optional, List, Set

# Project Idea: Find the Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## Part 1: Finding the determinant of a matrix

The determinant is very useful for so many things related to matrix algebra. One of its applications is in finding the eigenvalues of a matrix.

If I'm not lazy, I'll come back to write the explanation of how to find the determinant of a matrix. In the meanwhile, have [this](https://www.mathsisfun.com/algebra/matrix-determinant.html).

In [642]:
"""
Find the determinant of a given matrix (recursively)

Param: A, input matrix
Returns: determinant of A
"""
def det(A:np.ndarray) -> float:
	size, size2 = A.shape
	if size != size2:
		raise ValueError("Matrix A must be square")

	if size == 2:
		return A[0, 0]*A[1, 1] - A[1, 0]*A[0, 1]

	else:
		accumulator = 0
		sign = -1
		for i in range(size):
			sub_matrix = np.array([ [ A[row, col] for col in range(size) if col != i ] for row in range(1, size) ])
			accumulator += (sign**i) * A[0, i]*det(sub_matrix)
		return accumulator

In [643]:
# np.random.seed(123)
# test_matrix = np.random.randint(1, 10, size=(2, 2))
# print("np det:", np.linalg.det(test_matrix))
# print("my det:", det(test_matrix))

# test_matrix = np.random.randint(1, 20, size=(3, 3))
# print(test_matrix)
# print("np det:", np.linalg.det(test_matrix))
# print("my det:", det(test_matrix))

## Part 2: Fancy root finding

In Linear Algebra B, we learned that we can find the eigenvalues of a matrix by finding the roots or values of $\lambda$ such that $\det(A - \lambda I) = 0$. 

To find the roots, we will use Newton's method because it's faster so it should give us slightly better performance since we might need to find multiple roots.

In [644]:
"""
Find the derivative of a given point of function f using the centering method

Params:
* f: function to be used
* x: given point to find the derivative

Returns: slope at given point
"""
def derivative(f:Callable[[float], float], x:float):
	h = 0.001
	return (f(x+h) - f(x-h)) / 2/h

In [645]:
"""
Fancier version of Newton's method

Params:
* f: the function such that we want to find the point that f(x) = 0
* l: initial left bound
* r: initial right bound

Returns: a root in given bound rounded to 6 digits or None if there's no root or takes too long
"""
def newton(f:Callable[[float], float], x0) -> Optional[float]:
	BAD_TIME = 50
	x = x0
	c = 0
	while f(x) != 0 and c < BAD_TIME:
		# print(f(x))
		x = x - (f(x) / derivative(f, x))
		c += 1
	
	if f(x) == 0:
		return round(x, 6)
	elif abs(f(x)) < 1e-10:
		return round(x, 6)
	else:
		return None

In [646]:
newton(lambda x : np.cos(x) - x, 0)

0.739085

In [647]:
"""
For finding multiple roots within the given bound, using the shotgunning along with Newton's method

Params:
* f: the function we want to find roots for f(x) = 0
* l: left bound
* r: right bound
* n: how many initial points do we want

Returns: list of roots rounded to 3 digits if any
"""
def find_roots(f:Callable[[float], float], l:float, r:float) -> List[float]:
	d = 1
	n = (r-l)/d
	roots = set()
	for i in range(int(n)):
		try:
			res = newton(f, l + d*i)
		except ZeroDivisionError:
			res = None
		if res != None:
			roots.add(res)
	return roots

In [648]:
find_roots(lambda x: x**2+x-5, -100, 100)

{-2.791288, 1.791288}

## Part 3: Finding the eigenvalues

This is the first difficult part.

While it is quite easy to find the values of $\lambda$ such that it satisfies the condition $\det(A - \lambda I) = 0$, it still requires manual input of the initial bound, which is not good because we want to automate this.

Thus, we need to come up with a way to decide on the initial bound automatically

In [649]:
"""
Creating the function to calculate the det(A-lmbda*I) = 0

Param: matrix A
Returns: function that calculates det(A-lambda*I) for given lambda
"""
def zero_det(A:np.ndarray) -> Callable[[float], float]:
	def func(lmbda:float) -> float:
		size, _ = A.shape
		return det(A - lmbda*np.identity(size))
	return func

In [650]:
test_matrix = np.array([[5., 8., 16.],
						[4., 1., 8.],
						[-4., -4., -11.]])
print(test_matrix)
print(d := det(test_matrix))
find_roots(zero_det(test_matrix), -abs(d)*3, abs(d)*3)

[[  5.   8.  16.]
 [  4.   1.   8.]
 [ -4.  -4. -11.]]
9.0


{-3.0, 1.0}

In [651]:
np.random.seed(92)
test_matrix = np.random.randint(0, 10, size=(3, 3))
print(test_matrix)
print(d := det(test_matrix))
find_roots(zero_det(test_matrix), -abs(d)*3, abs(d)*3)

[[3 2 6]
 [7 1 4]
 [9 0 0]]
18


{-6.127891, -0.282168, 10.410059}